# DLISIO for ACE 2020

## Importing

In [ ]:
import dlisio

In [ ]:
%matplotlib inline

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import numpy.lib.recfunctions as rfn
import re
import tempfile
import sys
import hvplot.pandas
import holoviews as hv
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
hv.extension('bokeh', logo=None)
from IPython.display import display, HTML
from bokeh.models.tools import LassoSelectTool, BoxSelectTool  # tools=['hover']
box_select = BoxSelectTool()
lasso_select = LassoSelectTool()
import panel as pn
pn.extension()
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 60%; }
</style>
"""))

## We will use a file from the Volve Dataset:

In [ ]:
filepath = r"\WL_RAW_CAL-DEN-GR-NEU-REMP_MWD_1.DLIS"

In [ ]:
file = dlisio.dlis.load(filepath)

In [ ]:
file.describe()

## What is the origin metadata?

In [ ]:
for d in file:
    for origin in d.origins:
        print(origin.describe())

## What is the curve inventory?

In [ ]:
for d in file:
    for frame in d.frames:
        print(frame.describe())

## What curve information is available?

In [ ]:
for d in file:
    for frame in d.frames:
        for channel in frame.channels:
            print(channel.describe())

## Let's get the actual well log data now...

### First for a well log by name either via a find or explicit...

In [ ]:
for d in file:
    search_log = d.find('CHANNEL','PEF')
    for channel in search_log:
        print(channel.name)
        print(channel.curves())

In [ ]:
for d in file:
     for frame in d.frames:
        for channel in frame.channels:
            if channel.name == 'PEF':
                print(channel.curves())

### Then for all well logs in the entire file...

In [ ]:
for d in file:
    for frame in d.frames:
        curves = frame.curves()


curves

## Getting curves into pandas dataframes.

In [ ]:
for d in file:
    frame_count = 0
    for frame in d.frames:
        if frame_count == 0:
            curves = frame.curves()
            frame_count = frame_count + 1

In [ ]:
curves

In [ ]:
curves['HAZI']

In [ ]:
for d in file:
    frame_count = 0
    for frame in d.frames:
        if frame_count == 0:
            curves = frame.curves()
            df = pd.DataFrame(curves[['TDEP','DEVI', 'HAZI', 'PEF']], index=curves[frame.index])
            frame_count = frame_count + 1

In [ ]:
df

## Visualize

In [ ]:
df['TDEP_ft'] = df.loc[:,'TDEP'] * 0.0083333  #Convert to feet from 0.1in
#curve_list = list(df.columns)
#curve_list.remove('TDEP_ft')
#curve_list = curve_list[0:15] #Take only the first 15 channels to display
curve_list = ['TDEP','DEVI', 'HAZI', 'PEF']

In [ ]:
def curve_plot(log, df, depthname):
  aplot = df.hvplot(x=depthname, y=log, invert=True, flip_yaxis=True, shared_axes=True,
                       height=600, width=300).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
  return aplot;

plotlist = [curve_plot(x, df=df, depthname='TDEP_ft') for x in curve_list]
well_section = hv.Layout(plotlist).cols(len(curve_list))
well_section